<a href="https://colab.research.google.com/github/GustavoRizzo/quorum-Gustavo-Albuquerque-Aplication/blob/main/Quorum_Gustavo_Albuquerque_Aplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quorum Chalange

## Participant: Gustavo Rizzo Soares Mendes de Albuquerque
###[LinkedIn](https://www.linkedin.com/in/)
###[Personal Page](https://gustavorizzo.github.io/sketch-curriculum-components/)


Firstly, I would like to mention that if this were not a test, I would choose to use Power BI instead of Python for this problem. This is because you already have the data processed and only need to aggregate it conveniently for visualization, and Power BI is well-suited for this role. Additionally, I am of the opinion that if a less technical tool meets the demands of the task, we should prefer it. It not only facilitates maintenance but also simplifies the hiring process.

Since this is a programming test, I will solve it using Python. I opted to use Google Colab, even though it's not my preferred IDE, because it ensures that the evaluator can run the application through the browser without relying on legislatoral machine configurations.


In [ ]:
# install libs
!pip install -q pandas

In [ ]:
# import libs
import pandas as pd

In [ ]:
# Read paths
BILLS_CSV = 'https://raw.githubusercontent.com/GustavoRizzo/quorum-Gustavo-Albuquerque-Aplication/main/bills.csv'
LEGISLATOR_CSV = 'https://raw.githubusercontent.com/GustavoRizzo/quorum-Gustavo-Albuquerque-Aplication/main/legislators.csv'
VOTES_CSV = 'https://raw.githubusercontent.com/GustavoRizzo/quorum-Gustavo-Albuquerque-Aplication/main/votes.csv'
VOTE_RESULTS_CSV = 'https://raw.githubusercontent.com/GustavoRizzo/quorum-Gustavo-Albuquerque-Aplication/main/vote_results.csv'
# Write paths
LEGISLATOR_SUPOORT_OPPOSE_COUNT_PATH = 'legislators-support-oppose-count.csv'
BILLS_PATH = 'bills.csv'

In [ ]:
# Data Types
bills_type = {'id': int, 'primary sponsor': int}
legislator_type = {'id': int, 'name': str}
votes_type = {'id': int, 'bill_id': int}
vote_results_type = {'id': int, 'legislator_id': int, 'vote_id': int, 'vote_type': int}

In [ ]:
# Load dataframe
df_bills = pd.read_csv(BILLS_CSV, dtype=bills_type)
df_legislator = pd.read_csv(LEGISLATOR_CSV, dtype=legislator_type)
df_votes = pd.read_csv(VOTES_CSV, dtype=votes_type)
df_vote_results = pd.read_csv(VOTE_RESULTS_CSV, dtype=vote_results_type)

In [ ]:
# Just for vote results I will change a little bit the be more readable
def map_vote_type(vote_type):
    if vote_type == 1:
        return 'YES'
    elif vote_type == 2:
        return 'NO'
    else:
        return 'N/A'

df_vote_results['vote_type_str'] = df_vote_results['vote_type'].apply(map_vote_type)

In [ ]:
# 1. How many bills did each legislator sponsor?
# Assuming that the legislator only vote once per bill
df_vote_results_grouped = (
    df_vote_results.groupby('legislator_id')
    .agg(
            num_supported_bills=('vote_type_str', lambda x: (x == 'YES').sum()),
            num_opposed_bills=('vote_type_str', lambda x: (x == 'NO').sum())
        )
    )

# Merge with legislator to get name column
df_legislator_supoort_oppose_count = pd.merge(df_legislator, df_vote_results_grouped, left_on='id', right_on='legislator_id', how='left')
# Beautifying the columns
df_legislator_supoort_oppose_count['num_supported_bills'] = df_legislator_supoort_oppose_count['num_supported_bills'].fillna(0).astype(int)
df_legislator_supoort_oppose_count['num_opposed_bills'] = df_legislator_supoort_oppose_count['num_opposed_bills'].fillna(0).astype(int)

# export to csv
df_legislator_supoort_oppose_count.to_csv(LEGISLATOR_SUPOORT_OPPOSE_COUNT_PATH, index=False)

In [ ]:
# 2. How many bills did each legislator sponsor that passed?
# Assuming that the legislator only vote once per bill
df_vote_results_grouped2 = (
    df_vote_results.groupby('vote_id')
    .agg(
            supporter_count =('vote_type_str', lambda x: (x == 'YES').sum()),
            opposer_count=('vote_type_str', lambda x: (x == 'NO').sum())
        )
    )

# Merge with votes, bills and legislator to get bill_id, title and primary_sponsor
df_merge1 = pd.merge(df_vote_results_grouped2, df_votes, left_on='vote_id', right_on='id', how='left')
df_merge2 = pd.merge(df_merge1, df_bills, left_on='bill_id', right_on='id', how='left')
df_merge3 = pd.merge(df_merge2, df_legislator, left_on='sponsor_id', right_on='id', how='left')

# Building the final dataframe
df_bills2 = df_merge3[['bill_id', 'title', 'supporter_count', 'opposer_count', 'name']]
# Beautifying the columns
df_bills2 = df_bills2.rename(columns={
    'bill_id': 'id',
    'name': 'primary_sponsor',
    })
# If column primary_sponsor is null, then fill with 'Unknown'
df_bills2['primary_sponsor'] = df_bills2['primary_sponsor'].fillna('Unknown')

# export to csv
df_bills2.to_csv(BILLS_PATH, index=False)

#Questions

## 1. Discuss your solution’s time complexity. What tradeoffs did you make?

As I mentioned in the comment before the code, if this were a real-world problem presented to me, I would solve it using Power BI. The development would be faster, and the visualization for the end user would be more user-friendly. If I were to address this problem as a data engineer, I would solve it using SQL. With two queries, we could meet the requirements with much less code.



## 2. How would you change your solution to account forfuture columns that might be requested, such as “Bill Voted On Date” or“Co-Sponsors”?

We wouldn't have any issues adding this information because I believe "Bill Voted On Date" would come from the Excel file Vote.csv, and the column "Co-Sponsors" from the Excel file Bills.csv. We are already merging these tables into df_merged1 and df_merged2, respectively. Therefore, it would be a matter of specifying the addition of these columns when constructing df_bills2.


## 3. How would you change your solution if instead ofreceiving CSVs of data, you were given a list of legislators or bills that you should generate a CSV for?



```python
# Create a list of names to filter
names_to_filter = ['Rep. John Yarmuth (D-KY-3)', 'Rep. John Yarmuth (D-KY-4)', 'Rep. Jamaal Bowman (D-NY-16)']
# Filter the dataframes
df_new = df_legislator_supoort_oppose_count[df_legislator_supoort_oppose_count['name'].isin(names_to_filter)]
df_new.head()
df_new2 = df_bills2[df_bills2['primary_sponsor'].isin(names_to_filter)]
df_new2.head()
```

## 4. How long did you spend working on the assignment?

I spend 2 hours working on the main code and a little bit more to answer the questions and bealtify that file.